# *Projet Python ENSAE*

## Tokenisation d'actifs immobiliers - partie valorisation

Le but de ce notebook principal est de **créer un modèle de valorisation des actifs immobiliers**. Pour cela, nous utilisons principalement les données extraites du **dataset Demande de Valeurs Foncières** (DVF) et des **données INSEE**. Nous nous sommes inspirés d'un projet d'Arnaud Hureaux que nous avons cherché ) approfondir et compléter.

Lien projet (source d'inspiration) : https://hureauxarnaud.medium.com/projet-estimateur-de-prix-dun-bien-immobilier-bas%C3%A9-sur-du-machine-learning-ae578fdacaca

# Plan du notebook

**Etape 0 : packages et importations de données**

**Etape 1 : preprocessing**

* 1.1. Importation du dataset DVF
* 1.2. Visualisation des données DVF
* 1.3. Création du dataset final DVF
* 1.4. Valeurs extrêmes dans le dataset DVF

**Etape 2 : feature engineering**

* 2.1. Choix des nouveaux features
* 2.2. Importation des nouveaux features
* 2.3. Jointure des deux bases de données

**Etape 3 : analyse descriptive**

* 3.1. Premières analyses
* 3.2. Analyse de la répartition des prix de vente
* 3.3. Analyse de la corrélation entre surface et prix de vente
* 3.4. Premières intuitions sur l'analyse à l'échelle d'un département

**Etape 4 : modélisation**

* 4.1. Métriques et variables "dummies"
* 4.2. Modélisation sur tous les départements
* 4.3. Restriction au 75

# Etape 0 : packages et importations de données

In [4]:
# Importation des packages importants

!pip install nltk
!pip install unidecode
!pip install statsmodels
!pip install geopandas
!pip install folium

from nltk.metrics.distance import jaro_winkler_similarity
import unidecode
import statsmodels.api as sm
import geopandas as gpd
import folium

import numpy as np
import pandas as pd
from tqdm import tqdm
import requests
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

import zipfile
from urllib.request import urlopen
import shutil
import os

from sklearn.neighbors import BallTree
from scipy.spatial.distance import cdist
from scipy import stats

In [6]:
# Importation de données externes traitées dans un autre fichier
# Voire partie 2 (features) & le notebook external_data
!pip install import-ipynb
import import_ipynb
from external_data import retrieve_data

ImportError: cannot import name 'retrieve_data' from 'external_data' (external_data.ipynb)

# Etape 1 : preprocessing DVF

### 1.1. Importation du dataset DVF :

Nous importons le dataset « **Demandes de valeurs foncières** » (DVF), publié par la DGFiP, permet de connaître les transactions immobilières intervenues au cours des cinq dernières années sur le territoire métropolitain et les DOM-TOM, à l’exception de l’Alsace, de la Moselle et de Mayotte. Les données contenues sont issues des actes notariés et des informations cadastrales.

Fichiers 2017-2020 : https://files.data.gouv.fr/geo-dvf/latest/

In [211]:
# Les fichiers sont issus de https://files.data.gouv.fr/geo-dvf/latest/

name = "https://files.data.gouv.fr/geo-dvf/latest/csv/2021/full.csv.gz"
table = pd.read_csv(name, sep = ',')

# On ne travaille que sur les données du S1 2021, afin d'avoir des calculs moins coûteux en temps.
# Les transactions du S1 2021 représentent tout de même un dataset de 1 200 000 lignes...
# Pour travailler sur l'ensemble des données (2017-2021), il suffit d'enlever les guillemets ci-dessous.

"""
for year in range(2017, 2021):
    name = "https://files.data.gouv.fr/geo-dvf/latest/csv/" + str(year) + "/full.csv.gz"
    table = pd.concat([table, pd.read_csv(name, sep = ',')])

display("Taille de table :")
display(table.shape)
table.head()
"""

KeyboardInterrupt: 

### 1.2. Visualisation des données DVF

Il s'agit dans cette section de se faire **des premières intuitions sur les données**. 

En particulier, on se rend compte d'un **problème de preprocessing** : une transaction peut correspondre à plusieurs lignes avec la même valeur foncière sur chaque ligne. Autrement dit, **DVF affiche le même prix de vente global à chaque lot d'une même transaction**.

Pour observer cela, nous allons créer **un identifiant de transaction unique**.

In [ ]:
# Création d'une adresse générique

table['adresse_numero'] = table['adresse_numero'].fillna('0').astype(int)
table['adresse_suffixe'] = table['adresse_suffixe'].fillna(' ')
table['adresse_code_voie'] = table['adresse_code_voie'].fillna(' ')
table['adresse_nom_voie'] = table['adresse_nom_voie'].fillna(' ')
table['code_postal'] = table['code_postal'].fillna('0').astype(int)
table['nom_commune'] = table['nom_commune'].fillna(' ')

#Ajout de "\" pour que l'opération soit visible à l'écran en entier
table["adresse"] = table['adresse_numero'].astype(str) + ' ' + table['adresse_suffixe'] + ' ' + \
                table['adresse_code_voie'] + ' ' + table['adresse_nom_voie'] + ' ' + table['nom_commune'] + ' ' + \
                table['code_postal'].astype(str) + ' ' + 'France'

# Création d'un identifiant de transaction

# Pour identifier les doublons, l'adresse ne suffit pas : un bien peut avoir été vendu deux fois dans la même année
table["identifiant_transaction"] = table["adresse"].astype(str) + ' le ' + table["date_mutation"].astype(str)

In [ ]:
# Problème dans les données et vérification de la validité de l'identifiant de transaction :

display(table["identifiant_transaction"].loc[0])
display(table["identifiant_transaction"].loc[1])
display("Si l'identifiant de transaction est valide, alors True doit s'afficher :")
table["identifiant_transaction"].loc[0] == table["identifiant_transaction"].loc[1]

In [ ]:
# On constate, encore une fois, le problème relevé ci-dessus :

display("Nombre d'adresses uniques dans le DataFrame :")
display(len(table["adresse"].unique()))

display("Nombre d'identifiant_transaction uniques dans le DataFrame :")
display(len(table["identifiant_transaction"].unique()))

display("Nombre de lignes dans le DataFrame :")
display(len(table))

display("Nombre moyen de lignes par vente :")
np.round(len(table) / len(table["identifiant_transaction"].unique()), 2)

# Une vente correspond à plusieurs lignes, les informations sont donc diffusées dans ces lignes...

Dès lors, si on entraîne l'algorithme de pricing sur ce dataset, il sera **biaisé** : 
* d'une part, il associerait à une dépendance de 20 m2 le prix d'un appartement de 200 m2
* d'autre part, il ne prendrait pas en compte la plus-value apportée par un jardin à une maison, par une dépendance à un appartement, etc.

Il conviendra donc de **retravailler les données pour obtenir une seule ligne par transaction**.

**Le notebook "cleaning-dvf"** (lien : https://github.com/victor-kerros/tokenisation-immo/blob/main/cleaning-dvf.ipynb) revient en détail sur ce problème de preprocessing et effectue ce travail de nettoyage. Dans la mesure où cette solution de nettoyage est très coûteuse en temps, nous privilégions **une solution plus efficace** dans le cadre de ce projet : nous ne retenons que les transactions ne faisant l'objet que d'une seule ligne.

### 1.3. Création du dataset final DVF

D'abord, nous ne prenons que les colonnes suivantes :
- Date de vente/mutation
- Nature mutation (pour séparer les ventes en VEFA et les ventes classiques)
- Valeur foncière (prix de vente)
- Colonnes liées à l’adresse (pour nous permettre de localiser le bien)
- Adresse
- Code Postal
- Type local (maison/appartement/Local commercial/Dépendance etc)
- Surface réelle bâtie (nb de mètre carré du bien bâti)
- Surface terrain (nb de mètre carré du terrain associé au bien)

In [ ]:
# On crée le dataframe table_vf avec les seules colonnes qui nous intéressent
colonnes = ["date_mutation", "nature_mutation", "valeur_fonciere", "code_postal", 'type_local',
            'surface_reelle_bati', 'nombre_pieces_principales', 'nature_culture', 'surface_terrain', 'longitude', 
            'latitude', 'adresse', 'code_departement', 'identifiant_transaction']
table_vf = table[colonnes].copy()

# On agrège les types de cultures différents de NaN, sols, terrain à bâtir et  : on les renomme "culture"
culture_type = ['taillis simples', 'eaux', 'landes', 'taillis sous futaie', 'prés', 'terres', 'peupleraies', 
                'vignes', 'bois', 'vergers', 'carrières', 'futaies résineuses', 'pâtures', 'futaies feuillues', 
                'futaies mixtes', 'chemin de fer', 'oseraies', 'pacages', 'prés plantes', 'terres plantées', 
                'landes boisées', 'herbages', "prés d'embouche"]

for x in culture_type:
    table_vf.loc[table_vf["nature_culture"] == x, "nature_culture"] = "culture"

Ensuite, comme annoncé, nous ne retenons que les transactions ne faisant l'objet que d'une seule ligne pour créer "data" : le dataset final avec lequel nous allons travailler.

In [ ]:
# On ne retient que les transactions ayant fait l'objet d'une seule et unique ligne
table_vf_dup = table_vf.copy()
table_vf_uni = table_vf.copy()

# On récupère les indices des transactions dupliquées...
# i.e. les lignes où on retrouve un id de transaction utilisé ailleurs
dup_id = table_vf_dup.groupby('identifiant_transaction').size()
dup_id = dup_id[dup_id > 1]
dup_id = dup_id.reset_index()

table_vf_dup = table_vf_dup[table_vf_dup['identifiant_transaction'].isin(dup_id["identifiant_transaction"])]
table_vf_uni = table_vf_uni[~table_vf_uni['identifiant_transaction'].isin(dup_id["identifiant_transaction"])]

print("Taille de table_vf_uni (nombre de transactions non dupliquées) :")
print(table_vf_uni.shape)

# La solution la plus efficace, pour éviter un nettoyage trop coûteux en temps...
# est de ne retenir que les transactions non dupliquées.
data = table_vf_uni
data = data.reset_index().drop("index", axis = 1)

# Visualisation de data
display("Les trois premières lignes de data :")
display(data.head(3))

### 1.4. Valeurs extrêmes dans le dataset DVF

On s'intéresse aux **valeurs extrêmes dans le dataset**. En effet, afin d'avoir un entraînement fiable, nous devons les enlever (sinon, il y aura trop de bruit).

In [ ]:
def boxplot_display(data):

    fig, axs = plt.subplots(1,4)
    fig.suptitle("Boxplot des variables d'intérêt :")

    axs[0].boxplot(data[data['valeur_fonciere'].notna()]['valeur_fonciere'])
    axs[0].set(title = "Valeurs foncières")

    axs[1].boxplot(data[data['surface_reelle_bati'].notna()]['surface_reelle_bati'])
    axs[1].set(title = "Surface bati")

    axs[2].boxplot(data[data['nombre_pieces_principales'].notna()]['nombre_pieces_principales'])
    axs[2].set(title = "Nbre pièces")

    axs[3].boxplot(data[data['surface_terrain'].notna()]['surface_terrain'])
    axs[3].set(title = "Surface terrain")

    fig.tight_layout()

    plt.show()

In [ ]:
boxplot_display(data)

In [ ]:
ax = sns.boxplot(x = "valeur_fonciere", y = "type_local", data = data)

On constate sur ces boxplots que le dataset présente des valeurs extrêmes **particulièrement hautes** dans les quatre variables d'intérêt. 

En particulier, les locaux industriels, commerciaux et assimilés ont des valeurs extrêmes très importantes.

On observe également des valeurs extrêmes **particulièrement basses**, comme le montrent les cellules ci-dessous.

In [ ]:
display(len(data["valeur_fonciere"]))
valeurs = [1.0, 10.0, 1000.0, 5000.0, 10000.0]
for i in valeurs:
    display("Le nombre de valeurs foncières inférieures ou égales à "+str(i)+" est de:")
    display(sum(data["valeur_fonciere"] <= i))

In [ ]:
display(len(data["surface_reelle_bati"]))
valeurs = [1.0, 5.0, 10.0, 30.0]
for i in valeurs:
    display("Le nombre de surfaces bati intérieures ou égales à "+str(i)+" est de:")
    display(sum(data["surface_reelle_bati"] <= i))

On constate que bien qu'il n'y ait pas de valeurs foncières nulle, il y en a **de nombreuses qui sont inférieures ou égales à 10**.
On considère qu'**une transaction est crédible lorsque la valeur foncière est supérieure à 5000** (en-dessous, il s'agit d'une vente qui ne nous intéresse pas).

De même, pn considère qu'**une transaction est crédible lorsque la surface du bâtiment est supérieure à 10 m2**.

In [ ]:
# D'abord, on écarte les valeurs foncières inférieures à 5000 et les surface_reelle_bati inférieures à 10 m2

data = data[data["valeur_fonciere"] > 4999]

data = data[data["surface_reelle_bati"] > 9]

In [ ]:
# Il n'y a plus de Dépendances dans le dataset...

data["type_local"].unique()

In [ ]:
data.drop(["code_postal", "longitude", "latitude"], axis = 1).describe()

Par ailleurs, on observe avec le describe ci-dessus que **les écart-types sont très importants par rapport aux moyennes**, surtout pour les variables "valeur_fonciere", "surface_reelle_bati" et "surface_terrain".
Donc, **on enlève les valeurs trop hautes** : ici, cela consiste à *enlever les valeurs dont l'écart à la moyenne en valeur absolue est supérieure à 4 fois l'écart-type*.

In [ ]:
# Pour valeur_fonciere :
data = data[~(np.abs(data['valeur_fonciere'] - data['valeur_fonciere'].mean()) > (4 * data['valeur_fonciere'].std()))]

# Pour surface_reelle_bati :
data = data[~(np.abs(data['surface_reelle_bati'] - data['surface_reelle_bati'].mean()) > (4 * data['surface_reelle_bati'].std()))]

# Pour surface_terrain :
data = data[~(np.abs(data['surface_terrain'] - data['surface_terrain'].mean()) > (4 * data['surface_terrain'].std()))]

In [ ]:
data.drop(["code_postal", "longitude", "latitude"], axis = 1).describe()

In [ ]:
boxplot_display(data)

In [ ]:
ax = sns.boxplot(x = "valeur_fonciere", y = "type_local", data = data)

**Ces boxplots correspondent davantage avec la réalité** (médiane des ventes autour de 150 000 euros).

# Etape 2 : feature engineering

### 2.1. Choix des nouveaux features :

Afin d'**avoir un modèle performant**, nous allons **ajouter d'autres features**. En effet, la localisation, le prix, le nombre de pièces et la surface ne donnent pas assez d'informations pour bien valoriser un bien immobilier.

Selon BSI Economics, **trois facteurs** qui influencent les prix de l'immobilier :

- **l'environnement économique et financier** (offre et demande de logement, contexte économie, structure des marchés immobiliers, logements neufs et anciens, démographie, taux de logements vacants) ;

- **les conditions d'emprunts** (taux variables/fixes, maturités des prêts, ratio emprunt sur valeur, répartition des crédits, solvabilité, système de garantie) ;

- **l'environnement fiscal** (mesures fiscales incitatives à la location, distorsion fiscale des locataires vers les propriétaires).

En particulier, **un agent immobilier** observe, par exemple, les features suivant :
* prix au m2 du quartier (le projet qui nous a inspiré utilise l'algorithme BallTree pour l'obtenir), 
* PIB du département (mesure de l'activité économique des alentours), 
* densité du département (mesure de l'urbanisation, de l'activité), 
* variation de population non naturelle (mesure de l'attractivité),
* mois de la transaction (saisonnalité des prix dans certaines régions touristiques), 
* taux de logements vacants dans la région (mesure de l'attractivité).

### 2.2. Importation des nouveaux features :

In [ ]:
# Creation de la variable prix_m2 :
data['prix_m2'] = data['surface_terrain'] / data['valeur_fonciere']

Ces features ont été **générées dans le notebook "external-data"**. Nous invitons le lecteur à s'y référer afin d'apprécier la manière dont ils ont été importés puis préprocessés.

In [ ]:
# Importation des données externes INSEE :

#Old via excel
#file_path = "https://github.com/victor-kerros/tokenisation-immo/raw/main/external-data.csv"
#external_data = pd.read_csv(file_path, sep = ",")

external_data = retrieve_data()
external_data.head(3)

In [ ]:
list_numbers = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
data["code_departement"] = data["code_departement"].apply(
    lambda dep: str(dep) if str(dep) not in list_numbers else "0"+str(dep))
data.head(3)

In [ ]:
dep_vente = data["code_departement"].unique()
dep_commun = external_data["code_dep"].unique()

print(f"Département présents dans les ventes mais pas dans nos données complémentaires : \
      {[code_dep for code_dep in dep_vente if code_dep not in dep_commun]}")
print(f"Département présents dans les données complémentaires mais pas dans les ventes : \
{[code_dep for code_dep in dep_commun if code_dep not in dep_vente]}")
print(f"Nombre de départements uniques dans notre table générale : {len(data['code_departement'].unique())}.")

Les Landes (40) sont présentes dans les ventes mais **pas dans nos données complémentaires** (à cause du df chômage)...
Par ailleurs, il n'y a que **97 départements uniques** dans notre table générale...

In [ ]:
display("Nombre de lignes correspondant aux Landes qui vont être perdues")
display(data[data["code_departement"] == '40']["code_departement"].count())
display("Longueur avant inner join :")
len_pre_join = data.shape[0]
display(data.shape[0])

### 2.3. Jointure des deux tables de données :

In [ ]:
data = data.set_index("code_departement").join(external_data.set_index("code_dep"), how = "inner", rsuffix = "jo")
data = data.reset_index().rename(columns = {"index": "code_departement"})
display("Longueur après inner join :")
display(data.shape[0])
len_post_join = data.shape[0]
display("Lignes perdues pendant la jointure (du fait de l'inner join) :")
display(abs(len_post_join - len_pre_join))

L'inner join avec les autres features nous fait perdre les 1799 lignes correspondant aux ventes dans le département 40, comme on pouvait s'y attendre.

La base de donnée est bien cleanée avec **les nouveaux features ajoutés**.

# Etape 3 : analyse descriptive

### 3.1. Premières analyses :

In [ ]:
# Visualisation des lignes :
data.head(2)

In [ ]:
# Visualisation des valeurs uniques dans les colonnes "nature_culture" et "type_local" :

display(data["nature_culture"].unique())
display(data["type_local"].unique())

In [ ]:
nb_maisons = list(data["type_local"]).count("Maison")
print(f"Le nombre de maisons dans le dataset est : {nb_maisons}.")

nb_appartement = list(data["type_local"]).count("Appartement")
print(f"Le nombre d'appartments dans le dataset est : {nb_appartement}.")

nb_local = list(data["type_local"]).count("Local industriel. commercial ou assimilé")
print(f"Le nombre de 'Local industriel. commercial ou assimilé' dans le dataset est : {nb_local}.")

### 3.2. Analyse de la répartition des ventes :

In [ ]:
# Répartition des prix de vente :

plt.hist(data["valeur_fonciere"].values, bins = 100)
plt.title('Répartition des valeurs foncières')

Il y a encore des valeurs extrêmes qui **gênent la visualisation**. Nous allons **renlever les valeurs extrêmes** pour pouvoir mieux observer les données (nous enlevons les valeurs foncières dont l'écart à la moyenne est supérieur à 2,5 fois l'écart type). 

**Attention** : nous n'enlevons pas ces données de notre dataset principal (pour cela, nous créons data_aux).

On constate que la distribution est celle d'une **loi exponentielle**.

N.B. : nous utilisons également **le package seaborn** qui offre de bonnes opportunités de visualisation.

In [ ]:
data_aux = data[~(np.abs(data['valeur_fonciere'] - data['valeur_fonciere'].mean()) > (2.5 * data['valeur_fonciere'].std()))]

sns.displot(data_aux, 
            x = "valeur_fonciere", 
            bins = 50)
plt.xticks(rotation = 45)
plt.show()

Distinguons les **types de biens** désormais :

In [ ]:
data_aux = data_aux.drop(data_aux.loc[data_aux['type_local'] == 'Local industriel. commercial ou assimilé'].index)

sns.displot(data = data_aux,
            x = "valeur_fonciere",
            hue = "type_local",
            element = "step")

plt.xticks(rotation = 45)
plt.show()

On constate sur ce graphique que, dans ce dataset ("data_aux"), **les valeurs foncières des maisons sont sont en moyenne légèrement plus élevées que celles des appartements**. Cependant, cela est lié au fait que **nous avons tronqué le dataset**.

In [ ]:
print(f"Prix moyen des maisons : {np.round(data.loc[data['type_local'] == 'Maison']['valeur_fonciere'].mean())}")
print(f"Prix moyen des appartements : {np.round(data.loc[data['type_local'] == 'Appartement']['valeur_fonciere'].mean())}")

print(f"Ecart-type des prix des maisons : {np.round(data.loc[data['type_local'] == 'Maison']['valeur_fonciere'].std())}")
print(f"Ecart-type des prix des appartements : {np.round(data.loc[data['type_local'] == 'Appartement']['valeur_fonciere'].std())}")

En effet, le prix moyen des appartements est très légèrement plus élevé que le prix moyen des maisons mais **l'écart-type des prix des appartements est sensiblement plus élevé que celui des maisons**.

Cela s'interprète par le fait que **les appartements dans les centres grandes villes sont largement majoritaires et très chers** (il y a peu de maisons dans ces zones, s'il y en a elles sont très très chères mais il y a trop peu pour que cela ait un impact sur la variance).

### 3.3. Corrélation entre valeur foncière et surface :

In [ ]:
# Représentation des transactions d'appartements et maisons : valeur foncière en fonction de la surface

data_aux = data_aux.drop(data_aux.loc[data_aux['type_local'] == 'Dépendance'].index)

ax = sns.relplot(x = "surface_reelle_bati", 
            y = "valeur_fonciere", 
            hue = "type_local", 
            data = data_aux.sample(2000))
ax.set(title = "Relation entre valeur foncière et surface du bâti",
       xlabel = "Surface réelle bâti", ylabel = "Valeur foncière")
plt.show()

On ne constate **pas une corrélation très marquée**... En effet, il faudrait **distinguer la localisation** du bien (Paris vs. petite ville vs. campagne).

### 3.4. Premières intuitions sur l'analyse à l'échelle départementale :

In [ ]:
# Représentation des transactions d'appartements et maisons pour Paris

data_aux_paris = data.drop(data.loc[data['code_departement'] != "75"].index)

data_aux_paris = data_aux_paris.drop(data_aux_paris.
                                     loc[data_aux_paris['type_local'] =='Local industriel. commercial ou assimilé'].index)

ax = sns.relplot(x = "surface_reelle_bati", 
            y = "valeur_fonciere", 
            hue = "type_local", 
            data = data_aux_paris.sample(10000))
ax.set(title = "Relation entre valeur foncière et surface du bâti",
       xlabel = "Surface réelle bâti", ylabel = "Valeur foncière")
plt.show()

**Observation de la corrélation** avec lmplot :

In [ ]:
ax = sns.lmplot(x = "surface_reelle_bati", 
            y = "valeur_fonciere", 
            hue = "type_local", 
            data = data_aux_paris.sample(10000))
ax.set(title = "Relation entre valeur foncière et surface du bâti",
       xlabel = "Surface réelle bâti", ylabel = "Valeur foncière")
plt.show()

Pour la ville de Paris, la corrélation est très marquée !

In [ ]:
# Représentation des transactions d'appartements et maisons pour le Finistère

data_aux_finistere = data.drop(data.loc[data['code_departement'] != "29"].index)
data_aux_finistere = data_aux_finistere.drop(data_aux_finistere.loc[data_aux_finistere['type_local'] == 'Local industriel. commercial ou assimilé'].index)
display(data_aux_finistere.shape)

ax = sns.lmplot(x = "surface_reelle_bati", 
            y = "valeur_fonciere", 
            hue = "type_local", 
            data = data_aux_finistere.sample(200))
ax.set(title = "Relation entre valeur foncière et surface du bâti",
       xlabel = "Surface réelle bâti", ylabel = "Valeur foncière")
plt.show()

### 3.5. Analyse par département :

In [ ]:
# Les départements où les prix moyens des transaction sont les plus élevés :

ranking_dpt_mean_price = data.groupby("code_departement")["valeur_fonciere"].mean().sort_values(ascending = False)

print("Les cinq départements où les prix moyens des transaction sont les plus élevés :")
display(pd.DataFrame(ranking_dpt_mean_price).head(5))

print("Visualisation graphique des 20 départements où les prix moyens des transaction sont les plus élevés :")
fig, axs = plt.subplots()
axs.set(title = "Prix moyen de la transaction par départements")
axs.bar(x = [str(dep) for dep in ranking_dpt_mean_price.head(20).index.unique()],
            height = ranking_dpt_mean_price.head(20) )
plt.xticks(rotation = 45)
plt.show()

Sans surprise, **les départements de la région parisienne sont en tête du classement** des départements où les prix moyens des transaction sont les plus élevés.

In [ ]:
# Les départements où il y a eu le plus de transactions :

ranking_dpt_nb = data.groupby("code_departement")["valeur_fonciere"].count().sort_values(ascending = False)
print("Les cinq départements où il y a eu le plus de transactions :")
display(pd.DataFrame(ranking_dpt_nb).head(5))

print("Visualisation graphique des 20 départements où il y a eu le plus de transactions :")
fig, axs = plt.subplots()
axs.set(title = "Nombre de transaction par départements")
axs.bar(x = [str(dep) for dep in ranking_dpt_nb.head(20).index.unique()],
            height = ranking_dpt_nb.head(20) )
plt.xticks(rotation = 45)
plt.show()

In [ ]:
# Les départements où les nombres moyens de m2 sont les plus élevés :

ranking_dpt_nb_m2 = data.groupby("code_departement")["surface_reelle_bati"].mean().sort_values(ascending = False)
print("Les cinq départements où les nombres moyens de m2 sont les plus élevés :")
display(pd.DataFrame(ranking_dpt_nb_m2).head(5))

print("Visualisation graphique des 20 départements où les nombres moyens de m2 sont les plus élevés :")
fig, axs = plt.subplots()
axs.set(title = "Nombre de transaction par départements")
axs.bar(x = [str(dep) for dep in ranking_dpt_nb_m2.head(20).index.unique()],
            height = ranking_dpt_nb_m2.head(20) )
plt.xticks(rotation = 45)
plt.show()

### 3.6. QQ-plot :

Les QQ-plot pour la variable "valeurs foncières" correspondent à ceux d'une **distribution exponentielle**, ce qui confirme notre intuition sur la répartition des valeurs foncières.

In [ ]:
sm.qqplot(data["valeur_fonciere"])
plt.show()

In [ ]:
stats.probplot(data["valeur_fonciere"], plot = plt)

### 3.7. Représentation cartographique :

In [ ]:
sub_data = data[data['code_departement'] == "75"]
sub_data = sub_data[sub_data['longitude'].notna()]
sub_data = sub_data[sub_data['latitude'].notna()]
sub_data = sub_data.sample(1000).reset_index()

In [ ]:
paris = folium.Map(location = [48.856578, 2.351828], zoom_start = 12)
coord = [float(sub_data['latitude'][0]), float(sub_data['longitude'][0])]
id_ = str(sub_data["identifiant_transaction"][0]) + "au prix : " + str(sub_data["valeur_fonciere"][0])
folium.CircleMarker(coord, 
                    popup = id_, 
                    radius = 2).add_to(paris)

for index in tqdm(range(sub_data.shape[0]), desc = "Progression"):
    coord = [float(sub_data["latitude"][index]), float(sub_data["longitude"][index])]
    id_ = str(sub_data["identifiant_transaction"][index]) + " au prix : " + str(sub_data["valeur_fonciere"][index])
    folium.CircleMarker(coord, 
                        popup = id_, 
                        radius = 2).add_to(paris)
    
paris

In [ ]:
sub_data = data[data['longitude'].notna()]
sub_data = sub_data[sub_data['latitude'].notna()]
sub_data = sub_data.sample(10000).reset_index()

In [ ]:
france = folium.Map(location = [46.227638, 2.213749], zoom_start = 5)
coord = [float(sub_data['latitude'][0]), float(sub_data['longitude'][0])]
id_ = str(sub_data["identifiant_transaction"][0]) + "au prix : " + str(sub_data["valeur_fonciere"][0])
folium.CircleMarker(coord, 
                    popup = id_, 
                    radius = 2).add_to(france)

for index in tqdm(range(sub_data.shape[0]), desc = "Progression"):
    coord = [float(sub_data["latitude"][index]), float(sub_data["longitude"][index])]
    id_ = str(sub_data["identifiant_transaction"][index]) + " au prix : " + str(sub_data["valeur_fonciere"][index])
    folium.CircleMarker(coord, 
                        popup = id_, 
                        radius = 2).add_to(france)
    
france

In [ ]:
# lien tuto folium : https://fxjollois.github.io/cours-2016-2017/analyse-donnees-massives-tp9.html

# Etape 4 : modélisation

**Imports pour la modélisation**

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

from random import seed

#### Fixer une graine pour l'aléatoire

In [ ]:
seed(10)

*Noter qu'il aurait fallu faire de la cross-validation pour éviter la variabilité*

### 4.1 Métriques et variables "dummies"

**Concernant le choix des métriques d'évaluation des modèles** 
Ici on cherche à prédire le plus précisemment possible la valeur du logement à partir de ses caractéristiques.
Les métriques utilisées sont :
- *R2*
- *Mean_squarred_error* "expected value of the squarred error"
- *MAPE*: mean absolute percentage error, mesure de la précision comme un pourcentage. Noter que cette métrique peut prendre des valeurs supérieures à 100, elle représente en quelque sorte un pourcentage moyen

Etant donné que ce qui nous intéresse dans ce projet est de prédire précisemment le prix du logement, la métrique **MAPE** 
semble la plus pertinente puisqu'elle nous donne une mesure en pourcentage de la précision de notre estimation (ou plutôt de l'écart en % de notre prédiction à la réalité).

In [ ]:
# Features selection

data_model = data
features = ['valeur_fonciere', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 
            'nature_culture', 'surface_terrain', 'code_departement', 'tx_natalite_2020_percent', 
            'nb_musees', 'chomage_2016', 'chomage_2017', 'chomage_2018', 'chomage_2019',
           'chomage_2020', 'chomage_2019', 'taux_vacances_2019']

data_model = data_model[features]

Etant donné que le modèle de randomforest de sklearn ne peut pas gérer seul les variables catégorielles, nous devons les 
transformer en dummies

In [ ]:
# Encoding categorical variables

# Nous avons découvert trop tard que faire des dummies n'est pas optimal pour les random forest

# local_type

data_model["encoded_local_type_m"] = [1 if local == "Maison" else 0 for local in data_model["type_local"]]
data_model["encoded_local_type_a"] = [2 if local == "Appartement" else 0 for local in data_model["type_local"]]
data_model["encoded_local_type_d"] = [3 if local == "Dépendance" else 0 for local in data_model["type_local"]]
data_model["encoded_local_type_l"] = [4 if local == "Local industriel. commercial ou assimilé" else 0 \
                                      for local in data_model["type_local"]]


list_name = ["type_local", "encoded_local_type_m", "encoded_local_type_a", "encoded_local_type_d", 
             "encoded_local_type_l"]

data_model["encoded_local_type"] = data_model.loc[:,list_name].sum(axis = 1)
data_model = data_model.drop(list_name, axis = 1)

# nature_culture

data_model["encoded_nature_culture_c"] = [1 if local == "culture" else 0 for local in data_model["nature_culture"]]
data_model["encoded_nature_culture_s"] = [2 if local == "sols" else 0 for local in data_model["nature_culture"]]
data_model["encoded_nature_culture_tb"] = [3 if local == "terrains a bâtir" else 0 for local in data_model["nature_culture"]]
data_model["encoded_nature_culture_j"] = [4 if local == "jardins" else 0 for local in data_model["nature_culture"]]
data_model["encoded_nature_culture_ta"] = [4 if local == "terrains d'agrément" else 0 for local in data_model["nature_culture"]]

list_name = ["nature_culture", "encoded_nature_culture_c", "encoded_nature_culture_s", 
             "encoded_nature_culture_tb", "encoded_nature_culture_j", "encoded_nature_culture_ta"]

data_model["encoded_nature_culture"] = data_model.loc[:,list_name].sum(axis = 1)

data_model = data_model.drop(list_name, axis = 1)

# code_departement

data_model["code_departement"] = [201 if code == "2A" else code for code in data_model["code_departement"]]
data_model["code_departement"] = [202 if code == "2B" else code for code in data_model["code_departement"]]

# departement en ordinal
# même si cela crée un biais

data_model["dep_order"] = data_model["code_departement"]
data_model["dep_order"] = data_model["dep_order"].apply(lambda dep: float(dep))

# enlever NaN (on les remplace par des 0, à justifier)
data_model = data_model.fillna(0)

data_model.head()

### 4.2. Entraînement de modèles sur tous les départements

**Régression linéaire simple**

In [ ]:
data_model_small = data_model.copy()
data_model.head()

In [ ]:
data_model_small = data_model_small[["surface_reelle_bati", "valeur_fonciere"]]
data_model_small.head()

In [ ]:
# Entraînement du modèle régression linéaire simple

training_data, test_data = train_test_split(data_model_small, test_size = 0.2)
y_train = training_data["valeur_fonciere"].values
X_train = training_data.drop(["valeur_fonciere"], axis = 1).values

model = LinearRegression(fit_intercept = True).fit(X_train, y_train)

In [ ]:
# Test du modèle régression linéaire simple

y_test = test_data["valeur_fonciere"].values
X_test = test_data.drop(["valeur_fonciere"], axis = 1).values
y_pred = model.predict(X_test)

In [ ]:
# Coefficients du modèle régression linéaire simple

print("Intercept: \n", model.intercept_)
print("Coefficients: \n", model.coef_)

In [ ]:
# Métriques du modèle régression linéaire simple

display("MAPE")
display(mean_absolute_percentage_error(y_test, y_pred))

display("MSE (e9)")
display(mean_squared_error(y_test, y_pred)/10**9)

display("R2")
display(r2_score(y_test, y_pred))

Ce modèle extrêmement simple explique seulement **6.5%** de la variance dans les données de test

In [ ]:
fig, axs = plt.subplots()
axs.scatter(X_test, y_test, color = "black", s = 1)
axs.plot(X_test, y_pred, color = "blue", linewidth = 3)
axs.set_xlabel("Surface réelle bâtie")
axs.set_ylabel("Valeur foncière")
plt.show()

Le graphique ci-dessus indique les vrais points de données (en noir) et la droite de la régression linéaire simple (en bleue)

**Régression linéaire multiple**

In [ ]:
# Entraînement du modèle régression linéaire multiple

training_data, test_data = train_test_split(data_model[["valeur_fonciere","surface_reelle_bati", "nombre_pieces_principales",
                                                       "surface_terrain", "encoded_local_type", "encoded_nature_culture",
                                                       "dep_order"]],
                                            test_size = 0.2)

y_train = training_data["valeur_fonciere"].values
X_train = training_data.drop(["valeur_fonciere"], axis = 1).values

model = LinearRegression(fit_intercept = True).fit(X_train, y_train)

In [ ]:
# Test du modèle régression linéaire multiple

y_test = test_data["valeur_fonciere"].values
X_test = test_data.drop(["valeur_fonciere"], axis = 1).values
y_pred = model.predict(X_test)

In [ ]:
# Coefficients du régression linéaire multiple
print("Intercept: \n", model.intercept_)
print("Coefficients: \n", model.coef_)

In [ ]:
# Métriques du modèle régression linéaire multiple

display("MAPE")
display(mean_absolute_percentage_error(y_test, y_pred))

display("MSE (e9)")
display(mean_squared_error(y_test, y_pred)/10**9)

display("R2")
display(r2_score(y_test, y_pred))

Ici, on monte à seulement **11 %** de variance expliquée en y rajoutant plein de variables, ce qui est assez peu

#### Random forest

In [ ]:
# Entraînement du modèle random forest

training_data, test_data = train_test_split(data_model[["valeur_fonciere","surface_reelle_bati", "nombre_pieces_principales",
                                                       "surface_terrain", "encoded_local_type", "encoded_nature_culture",
                                                       "dep_order"]], test_size = 0.2)

y_train = training_data["valeur_fonciere"].values
X_train = training_data.drop(["valeur_fonciere"], axis = 1).values

model = RandomForestRegressor(n_estimators = 30, min_samples_split = 5)

model.fit(X_train, y_train)

In [ ]:
# Test du modèle random forest

y_test = test_data["valeur_fonciere"].values
X_test = test_data.drop(["valeur_fonciere"], axis = 1).values
y_pred = model.predict(X_test)

In [ ]:
# Métriques du modèle random forest

display("MAPE")
display(mean_absolute_percentage_error(y_test, y_pred))

display("MSE (e9)")
display(mean_squared_error(y_test, y_pred)/10**9)

display("R2")
display(r2_score(y_test, y_pred))

On constate que la part de variance expliquée par le modèle monte à près de **51%**. Notre métrique MAPE reste cependant très élevée (>50%).

In [ ]:
schema = pd.DataFrame((np.abs(y_test - model.predict(X_test)) / y_test)) * 100

schema.hist(bins = 200)

plt.xlabel('Erreur en %')
plt.title('Distribution de l\'erreur de notre modèle')

Il semble que l'erreur de notre modèle soit tirée vers le haut par quelques points

#### Random forest incluant les données externes

In [ ]:
# Entraînement du modèle random forest avec les données externes

training_data, test_data = train_test_split(data_model.drop(["code_departement"], axis =1), test_size = 0.2)

y_train = training_data["valeur_fonciere"].values
X_train = training_data.drop(["valeur_fonciere"], axis = 1).values

model = RandomForestRegressor(n_estimators = 30, min_samples_split = 5)

model.fit(X_train, y_train)

In [ ]:
# Test du modèle random forest avec les données externes

y_test = test_data["valeur_fonciere"].values
X_test = test_data.drop(["valeur_fonciere"], axis = 1).values
y_pred = model.predict(X_test)

In [ ]:
# Métriques du modèle random forest avec les données externes

display("MAPE")
display(mean_absolute_percentage_error(y_test, y_pred))

display("MSE (e9)")
display(mean_squared_error(y_test, y_pred)/10**9)

display("R2")
display(r2_score(y_test, y_pred))

Ici, l'ajout de différentes données concernant les départements semblent peu pertinentes. En effet, elle ne fait agumenter le R2 que de 2% et fait même augmenter le MAPE

### 4.3 Entraînement restreint à Paris 75

#### Création du sous-jeu de données

In [ ]:
data_model_75 = data_model[['valeur_fonciere', 'surface_reelle_bati', 'nombre_pieces_principales',
       'surface_terrain', 'code_departement',  'encoded_local_type','encoded_nature_culture']]
data_model_75 = data_model_75[data_model_75["code_departement"] == "75"].drop("code_departement", axis = 1)

In [ ]:
data_model_75.columns

#### Régression linéaire simple

In [ ]:
# Entraînement du modèle régression linéaire simple pour Paris

training_data, test_data = train_test_split(data_model_75[["valeur_fonciere","surface_reelle_bati"]], test_size = 0.2)
y_train = training_data["valeur_fonciere"].values
X_train = training_data.drop(["valeur_fonciere"], axis = 1).values

model = LinearRegression(fit_intercept = True).fit(X_train, y_train)

In [ ]:
# Test du modèle régression linéaire simple

y_test = test_data["valeur_fonciere"].values
X_test = test_data.drop(["valeur_fonciere"], axis = 1).values
y_pred = model.predict(X_test)

In [ ]:
# Coefficients du modèle régression linéaire simple

print("Intercept: \n", model.intercept_)
print("Coefficients: \n", model.coef_)

In [ ]:
# Métriques du modèle régression linéaire simple

display("MAPE")
display(mean_absolute_percentage_error(y_test, y_pred))

display("MSE (e9)")
display(mean_squared_error(y_test, y_pred)/10**9)

display("R2")
display(r2_score(y_test, y_pred))

Cette régression linéaire toute simple explique ici 52 % de la variance ! Mais le MAPE est très élevée

In [ ]:
fig, axs = plt.subplots()
axs.scatter(X_test, y_test, color = "black", s = 1)
axs.plot(X_test, y_pred, color = "blue", linewidth = 3)
axs.set_xlabel("Surface réelle bâtie")
axs.set_ylabel("Valeur foncière")
plt.show()

On constate visuellement que pour Paris, une régression linéaire simple sur la surface relle batie permet d'expliquer une grand part de la variation dans les données (à part quelques données aberrantes)

#### Random forest

In [ ]:
# Entraînement du modèle random forest avec les données externes

training_data, test_data = train_test_split(data_model_75, test_size = 0.2)

y_train = training_data["valeur_fonciere"].values
X_train = training_data.drop(["valeur_fonciere"], axis = 1).values

model = RandomForestRegressor(n_estimators = 30, min_samples_split = 5)

model.fit(X_train, y_train)

In [ ]:
# Test du modèle random forest avec les données externes

y_test = test_data["valeur_fonciere"].values
X_test = test_data.drop(["valeur_fonciere"], axis = 1).values
y_pred = model.predict(X_test)

In [ ]:
# Métriques du modèle random forest avec les données externes

display("MAPE")
display(mean_absolute_percentage_error(y_test, y_pred))

display("MSE (e9)")
display(mean_squared_error(y_test, y_pred)/10**9)

display("R2")
display(r2_score(y_test, y_pred))

Utiliser random forest permet de faire monter un peu le R2 de 15% ici, mais le MAPE reste toujours très élevé

#### Conclusion principale de la modélisation
* L'utilisation de random forest nous permet de modéliser environ 50% de la variance dans les données
* Les données supplémentaires sur les départements ne semblent pas ici tèrs pertinentes
* Si l'on restreint le jeu de données à Paris, une régression linéaire simple semble très performante, elle pourrait l'être encore plus en enlevant quelques points aberrants